# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
vacation_df = pd.read_csv('../WeatherPy\output_data/WeatherPy.csv')
del vacation_df['Unnamed: 0']
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,arraial do cabo,-22.9661,-42.0278,21.92,64.0,35.0,2.29,BR,1.619843e+09
1,vaitupu,-21.2000,-175.2000,26.00,94.0,20.0,6.17,TO,1.619843e+09
2,vaini,18.2164,42.5053,19.00,42.0,40.0,2.89,SA,1.619843e+09
3,abha,-33.5906,26.8910,14.32,81.0,5.0,5.78,ZA,1.619843e+09
4,illoqqortoormiut,19.7297,-155.0900,26.00,65.0,75.0,3.60,US,1.619843e+09
...,...,...,...,...,...,...,...,...,...
614,hofn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,cam pha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,bacuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,torbat-e jam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
vacation_df.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,arraial do cabo,-22.9661,-42.0278,21.92,64.0,35.0,2.29,BR,1.619843e+09
1,vaitupu,-21.2000,-175.2000,26.00,94.0,20.0,6.17,TO,1.619843e+09
2,vaini,18.2164,42.5053,19.00,42.0,40.0,2.89,SA,1.619843e+09
3,abha,-33.5906,26.8910,14.32,81.0,5.0,5.78,ZA,1.619843e+09
4,illoqqortoormiut,19.7297,-155.0900,26.00,65.0,75.0,3.60,US,1.619843e+09
...,...,...,...,...,...,...,...,...,...
559,rio branco,43.6475,111.9767,10.08,24.0,8.0,4.74,CN,1.619844e+09
560,tommot,42.1859,127.4790,7.48,72.0,100.0,2.83,CN,1.619844e+09
561,port blair,64.2539,-15.2082,1.42,64.0,100.0,5.29,IS,1.619843e+09
562,san juan,35.2440,60.6225,20.73,48.0,100.0,3.43,IR,1.619844e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [78]:
gmaps.configure(api_key=g_key)
locations = vacation_df[["Lat","Lng"]]
humidity = vacation_df["Humidity"].astype(float)

In [79]:
locations = locations.dropna()
humidity = humidity.dropna()

In [80]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [81]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_weather = vacation_df.loc[(vacation_df["Wind Speed"] < 10) &
                                (vacation_df["Cloudiness"] == 0) &
                                (vacation_df["Max Temp"] >= 21) & 
                                (vacation_df["Max Temp"] <= 27)]

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,atuona,-42.8794,147.3294,23.00,38.0,0.0,1.54,AU,1.619843e+09
71,airai,-15.1961,12.1522,23.96,70.0,0.0,2.46,AO,1.619843e+09
77,oktyabrskoye,31.6100,34.7642,22.78,63.0,0.0,0.14,IL,1.619843e+09
142,mahebourg,16.6527,-14.9587,23.00,64.0,0.0,3.09,MR,1.619843e+09
191,isangel,22.8909,-109.9124,22.22,76.0,0.0,2.79,MX,1.619843e+09
207,sao filipe,35.0806,109.0897,25.68,22.0,0.0,2.86,CN,1.619843e+09
229,blackwater,13.2019,-16.7339,21.00,88.0,0.0,7.38,GM,1.619843e+09
273,shantipur,-32.2500,148.6167,25.00,38.0,0.0,2.57,AU,1.619843e+09
311,zarubino,20.5169,-13.0499,24.86,23.0,0.0,3.47,MR,1.619843e+09
312,halifax,29.9697,40.2064,24.00,17.0,0.0,2.57,SA,1.619843e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = " "
hotel_df

<ipython-input-82-146a1f237505>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,atuona,-42.8794,147.3294,23.00,38.0,0.0,1.54,AU,1.619843e+09,
71,airai,-15.1961,12.1522,23.96,70.0,0.0,2.46,AO,1.619843e+09,
77,oktyabrskoye,31.6100,34.7642,22.78,63.0,0.0,0.14,IL,1.619843e+09,
142,mahebourg,16.6527,-14.9587,23.00,64.0,0.0,3.09,MR,1.619843e+09,
191,isangel,22.8909,-109.9124,22.22,76.0,0.0,2.79,MX,1.619843e+09,
207,sao filipe,35.0806,109.0897,25.68,22.0,0.0,2.86,CN,1.619843e+09,
229,blackwater,13.2019,-16.7339,21.00,88.0,0.0,7.38,GM,1.619843e+09,
273,shantipur,-32.2500,148.6167,25.00,38.0,0.0,2.57,AU,1.619843e+09,
311,zarubino,20.5169,-13.0499,24.86,23.0,0.0,3.47,MR,1.619843e+09,
312,halifax,29.9697,40.2064,24.00,17.0,0.0,2.57,SA,1.619843e+09,


In [83]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
    "keyword" : "hotel"
    }
print("----------------------------------------------------------")
print("Search started, please wait. ")
print("----------------------------------------------------------")

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
  
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("----------------------------------------------------------")
print("----------------------------------------------------------")
print("Search completed successfully")
print("----------------------------------------------------------")
hotel_df

----------------------------------------------------------
Search started, please wait. 
----------------------------------------------------------
Retrieving Results for Index 50: atuona.
Closest hotel in atuona is Hotel Grand Chancellor Hobart.
----------------------------------------------------------
Retrieving Results for Index 71: airai.


C:\Users\joses\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest hotel in airai is Hotel Chik Chik Namibe.
----------------------------------------------------------
Retrieving Results for Index 77: oktyabrskoye.
Closest hotel in oktyabrskoye is Desert Gat.
----------------------------------------------------------
Retrieving Results for Index 142: mahebourg.
Closest hotel in mahebourg is La cour du fleuve.
----------------------------------------------------------
Retrieving Results for Index 191: isangel.
Closest hotel in isangel is The Cape, A Thompson Hotel.
----------------------------------------------------------
Retrieving Results for Index 207: sao filipe.
Closest hotel in sao filipe is Yibiju Business Hotel.
----------------------------------------------------------
Retrieving Results for Index 229: blackwater.
Missing field/result... skipping.
----------------------------------------------------------
Retrieving Results for Index 273: shantipur.
Closest hotel in shantipur is Quest Dubbo.
-------------------------------------------

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,atuona,-42.8794,147.3294,23.00,38.0,0.0,1.54,AU,1.619843e+09,Hotel Grand Chancellor Hobart
71,airai,-15.1961,12.1522,23.96,70.0,0.0,2.46,AO,1.619843e+09,Hotel Chik Chik Namibe
77,oktyabrskoye,31.6100,34.7642,22.78,63.0,0.0,0.14,IL,1.619843e+09,Desert Gat
142,mahebourg,16.6527,-14.9587,23.00,64.0,0.0,3.09,MR,1.619843e+09,La cour du fleuve
191,isangel,22.8909,-109.9124,22.22,76.0,0.0,2.79,MX,1.619843e+09,"The Cape, A Thompson Hotel"
207,sao filipe,35.0806,109.0897,25.68,22.0,0.0,2.86,CN,1.619843e+09,Yibiju Business Hotel
229,blackwater,13.2019,-16.7339,21.00,88.0,0.0,7.38,GM,1.619843e+09,
273,shantipur,-32.2500,148.6167,25.00,38.0,0.0,2.57,AU,1.619843e+09,Quest Dubbo
311,zarubino,20.5169,-13.0499,24.86,23.0,0.0,3.47,MR,1.619843e+09,Bab Sahara
312,halifax,29.9697,40.2064,24.00,17.0,0.0,2.57,SA,1.619843e+09,Raoum Inn Hotel


In [84]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Analysis
* From weather data obtained information can be use 0for multiple options, one is to plan vacations.
* Is important to drop NaN values before to make the heat layer. Latitudes must lie between -90 and 90.
* We can Narrow down the DataFrame to find our ideal weather conditions or any other criteria to match our destination.
* Then we can make another API request to find hotels, in this case close to our target destination. 
* This is a very good example of API´s powerfull use.